## Import dependencies

In [ ]:
# COCO related libraries
import sys
import os
sys.path.append(os.path.join("/mnt/samples/coco/"))  # To find local version
import coco
from pycocotools.coco import COCO

# MaskRCNN libraries
from mrcnn import utils, visualize
from mrcnn import model as modellib

# Misc
import cv2
import argparse as argparse
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import skimage.io
import scipy
import random
import json
%matplotlib inline 

## Constants

In [ ]:
# Number of classes in dataset. Must be of type integer
NUM_CLASSES = 9

# Relative path to .h5 weights file
WEIGHTS_FILE = "/mnt/Tomato_FounSeeds_Train.h5"

# Relative path to annotations JSON file
ANNOTATIONS_FILE = "/mnt/Annotation/Original Training/Train/annotations.json"

# Relative path to directory of images that pertain to annotations file
ANNOTATION_IMAGE_DIR = "/mnt/Annotation/Original Training/Train/"

# Relative path to the directory of images that you want to analyse
TEST_IMAGE_DIR = "/mnt/Images/"

## Additional Setup

In [ ]:
# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Select which GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

## Declare inference configuration

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Train on 1 image per GPU. Batch size is 1 (GPUs * images/GPU).
    # GPU COUNT is modified so that its actually the id of the GPU you want to use. For example, to use the 4th GPU
    # put 3 for GPU count (0 index)
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + NUM_CLASSES  # background + 1 (structure)

    # All of our training images are 300x300
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024
    
    # Matterport originally used resnet101, but I downsized to fit it on my graphics card
    BACKBONE = 'resnet50'

    # RPN Anchor Scales
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    
    # Changed to 512 because that's how many the original MaskRCNN paper used
    TRAIN_ROIS_PER_IMAGE = 200
    MAX_GT_INSTANCES = 114
    POST_NMS_ROIS_INFERENCE = 1000 
    POST_NMS_ROIS_TRAINING = 2000 
    
    DETECTION_MAX_INSTANCES = 114
    DETECTION_MIN_CONFIDENCE = 0.1
    
    STEPS_PER_EPOCH = 180
    VALIDATION_STEPS = 10

## Display configuration

In [ ]:
InferenceConfig().display()

## Create class to load dataset

In [ ]:
class CocoLikeDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """
    def load_data(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            if class_id < 1:
                print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
                return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
        
                     
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids

## Load dataset to get list of class names

In [ ]:
coco_dataset = CocoLikeDataset()
coco_dataset.load_data(ANNOTATIONS_FILE, ANNOTATION_IMAGE_DIR)
coco_dataset.prepare()
class_names = coco_dataset.class_names
class_id = coco_dataset.class_ids

## Build MaskRCNN Model

In [ ]:
model = modellib.MaskRCNN(mode = "inference", model_dir = MODEL_DIR, config = InferenceConfig())

## Load weights into model

In [ ]:
model.load_weights(WEIGHTS_FILE, by_name = True)

## Run detection model

In [ ]:
# Initiale Arrays
# Generate empty arrays for all the measurements we need
FileName = []
TomatoView = []

# Fruit Size
MaxHeight = []
CmHeight = []
MaxWidth = []
CmWidth = []
Area = []
CmArea = []

# Fruit Shape
ShapeIndex = []
ShapeTriangle = []
ShapeEccentric = []
IndexObovoid = []
IndexOvoid = []
AsymmetryHorizontal = []
AsymmetryVertical = []

# Fruit End Shape
AngleProx = []
AngleDist = []
BlockinessProx = []
BlockinessDist = []
ShoulderheightProx = []
ShoulderheightDist = []

# Fruit Shape Resemblance
Compactness = []
CompactnessEllipse = []
HeartP = []
HeartD = []
Rectangular = []

In [ ]:
# Image Type
def Image_Type(image_name):
    # Determine if the image contains a grid
    img = cv2.imread(image_name)

    # Converting image to grayscale and finding the edges of the objects
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray, 200, 200, apertureSize=3)

    # Finding the contours in the image to extract the grid
    cnts = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for c in cnts:
        cv2.drawContours(img, [c], 0, (0, 255, 0), 3)
    
    # if as sufficient amount of contours are found, this means that the image contains a grid. Else it doesn't.
    if (len(cnts)) > 3000:
        ImageType = "Grid"
    else:
        ImageType = "Non grid"
        
    return ImageType

In [ ]:
# Grid
def Grid_Scale(ImageType, image_name):
    # To find the scale of the image, if the image contains a grid (and thus ImageType = Grid).
    print(ImageType)
    img = cv2.imread(image_name)
        
    # Measuring the image dimensions
    height, width = img.shape[:2]
    imgarea = np.multiply(height, width)
    
    # Converting image to grayscale and finding the edges of the objects
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray, 200, 200, apertureSize=3)

    # Finding the contours in the image to extract the grid
    cnts = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # Initializing and storing the area of the squares of the grid (which we can find by finding 
    # only the area's that are not too small or too large) in an array
    sqarea = np.array([])

    for c in cnts:
        cv2.drawContours(img, [c], 0, (0, 255, 0), 3)
        area = cv2.contourArea(c)

        if imgarea/10000 < area < imgarea/500:
            sqarea = np.append(sqarea, area)
            
    # Measuring the mean area of the grid
    gridarea = np.mean(sqarea)
    
    # Calculating the amount of pixels per cm in the image and converting this to the amount of cm in each pixel.
    pixelspercm = np.sqrt(gridarea)
    pixelsize = 1/pixelspercm
            
    return pixelsize

In [ ]:
# Ruler
def Ruler_Scale(classids, roishape, masks):
    # To find the scale in an image containing a ruler.
    for i in range(0, roishape[0]):
        classID = classids[i]  # Finding all the classIDs, which indicate the different detections (masks) in the image.
        if classID == 1: # The classID 1 is the Ruler detection.
            ruler = masks[:, :, i]
            
            # sum over each row in the mask array will give the width of each row (thus each width in the ruler mask).
            width = []
            for j in range(ruler.shape[0]):
                width.append(sum(ruler[j,:])) 
                
            # The maximum width of the ruler will indicate the amount of pixels per cm in the image 
            # and converting this to the amount of cm in each pixel.
            rulerwidth = max(width)             
            pixelsize = 1/rulerwidth
            
    return pixelsize

In [ ]:
# Fruit Size
class Fruit_Size:
    """For the fruit size, the maximum height, maximum width and area are calculated. The output generates both pixel sizes 
    and sizes in centrimetres. Additionally, the first and last points of both the height and width are determined. 
    Also the width at mid height is determined. 
    The width at the proximal end and the distal end are also determined at 5 percent from the top or bottom, respectively. 
    This value can be adjusted by adjusting the distance_percentage variable. 
    """
    distance_percentage = 0.05 # amount of seperation of the proximal and distal points (5%), which can be adjusted if needed
    
    def __init__(self, mask, pixelsize):
        self.mask = mask
        self.pixelsize = pixelsize
        
    def max_height(self):
        # sum over the mask array will give amount of pixels in each column (thus each height in the mask).
        self.height = sum(self.mask) 
        self.maxheight = max(self.height)
        
        return self.height, self.maxheight
    
    def cm_height(self):
        # converting the height to centimetres, using the calculated scale.
        self.height, self.maxheight = self.max_height()
        self.cmheight = self.maxheight*float(self.pixelsize)
        
        return self.cmheight
           
    def max_width(self):
        # sum over each row in the mask array will give the width of each row (thus each width in the mask).
        self.width = []
        for j in range(self.mask.shape[0]):
            self.width.append(sum(self.mask[j,:]))
        self.maxwidth = max(self.width)
        
        return self.width, self.maxwidth
    
    def cm_width(self):
        # converting the width to centimetres, using the calculated scale.
        self.width, self.maxwidth = self.max_width()
        self.cmwidth = self.maxwidth*float(self.pixelsize)
        
        return self.cmwidth
       
    def pixel_area(self):
        # to obtain the area, summing to times is necessary to first sum over each column and then combine all columns.
        # This will give all pixels that are present in the mask (and thus the pixel area)
        self.area = sum(sum(self.mask))
        
        return self.area
        
    def cm_area(self):
        # Converting the area to centimetres by multiplying with the pixelsize squared.
        self.area = self.pixel_area()
        self.pixelarea = float(self.pixelsize)*float(self.pixelsize)
        self.cmarea = self.area*self.pixelarea
        
        return self.cmarea
    
    def extreme_points(self):
        # Finding the top (firstypoint) and bottom (lastypoint) of the mask
        for j in range(self.mask.shape[0]):    
            if self.width[j]!=0 and self.width[j-1]==0:
                self.firstypoint = j
                                        
            if self.width[j]!=0 and self.width[j+1]==0:
                self.lastypoint = j
        
        # Finding the most left (firstxpoint) and most right (lastxpoint) of the mask
        for j in range(self.mask.shape[1]):        
            if self.height[j]!=0 and self.height[j-1]==0:
                self.firstxpoint=j
                              
            if self.height[j]!=0 and self.height[j+1]==0:
                self.lastxpoint = j
                
        return self.firstypoint, self.lastypoint, self.firstxpoint, self.lastxpoint
    
    def prox_dist_width(self):
        # Getting the width at a pre-defined distance (distance_percentage) from the proximal and distal point.
        # distance_percentage can be adjusted at the start of the class.
        self.firstypoint, self.lastypoint, self.firstxpoint, self.lastxpoint = self.extreme_points()
        self.at_5percent = int(self.firstypoint+self.maxheight*self.distance_percentage)
        self.at_95percent = int(self.firstypoint+self.maxheight*(1-self.distance_percentage))          
        self.proxwidth = self.width[self.at_5percent]
        self.distwidth = self.width[self.at_95percent]
                        
        return self.proxwidth, self.distwidth  
    
    def center_width(self):
        # The width at mid height is determined by taking the top point of the mask and adding half of the maxheight 
        # to find the mid index. The width at this index is taken as midwidth.
        self.height, self.maxheight = self.max_height()
        self.width, self.maxwidth = self.max_width()
        self.firstypoint, self.lastypoint, self.firstxpoint, self.lastxpoint = self.extreme_points()
        self.midpoint = self.firstypoint+(self.maxheight/2)
        self.midwidth = self.width[int(self.midpoint)]
        
        return self.midwidth  

In [ ]:
# Fruit Shape
class Fruit_Shape():
    """ The fruit shape is described by the shape measurements: shape index, shape triangle, 
    shape eccentric (obovoid or ovoid measurement), shape asymmetry (horizontal and vertical) 
    and blockiness (proximal and distal). 
    """
    
    def __init__(self):
        # get all the measurements from different class(es) that we need to calculate the shape measurements.
        self.instance_size = Fruit_Size(mask, pixelsize)
        self.mask = mask
        self.width, self.maxwidth = self.instance_size.max_width()
        self.height, self.maxheight = self.instance_size.max_height()
        self.firstypoint, self.lastypoint, self.firstxpoint, self.lastxpoint = self.instance_size.extreme_points()
        self.proxwidth, self.distwidth = self.instance_size.prox_dist_width()
        self.midwidth = self.instance_size.center_width()
        
    def shape_index(self):
        # shape index is the ratio between the maximum height and maximum width and is added to the Shape Index array.
        self.shapeindex = self.maxheight/self.maxwidth
        
        return self.shapeindex
        
    def shape_triangle(self):
        # shape triangle is the ratio between the proximal and distal width. this is added to the Shape Triangle array.
        self.shapetriangle = self.proxwidth/self.distwidth
        
        return self.shapetriangle
    
    def asymmetry(self):
        # Determining the horizontal and vertical asymmetry.
        self.mdif = []
        self.ndif = []
        
        # take the vertical middle line (m) and half the width at that index (mj) to calculate the difference between 
        # those (mdif)
        for j in range(self.mask.shape[0]):         
            self.m = self.mask[j, int(self.firstxpoint+self.maxwidth*0.5)]
            self.mj = self.width[j]/2
            self.mdif.append(abs(self.m-self.mj))
        
        # take the horizontal middle line (n) and half the height at that index (nj) to calculate the difference between 
        # those (ndif)
        for j in range(self.mask.shape[1]):
            self.n = self.mask[int(self.firstypoint+self.maxheight*0.5),j]
            self.nj = self.height[j]/2
            self.ndif.append(abs(self.n-self.nj))
        
        # Calculate asymmetry and adding to their array.
        self.asymmetryhorizontal = sum(self.ndif)/(self.mask.shape[1])
        self.asymmetryvertical = sum(self.mdif)/(self.mask.shape[0])
        
        return self.asymmetryhorizontal, self.asymmetryvertical
    
    def find_y(self):
        # Determine the index of the widest width and normalize that to a value from 0 to 1 to find y. 
        for j in range(self.mask.shape[0]):
            if self.width[j] == self.maxwidth:
                self.widestindex = j
                self.y = j/(self.mask.shape[0])
        return self.widestindex, self.y
    
    def eccentric(self):
        # If y is greater than 0.5, calculate the ovoid measurement and add to the array
        self.widestindex, self.y = self.find_y()
        if self.y > 0.5:
            self.shapeeccentric = "Ovoid"
            self.indexobovoid = 0
            self.indexovoid = 4*(self.y-0.5)
        else:
        # If y is smaller than 0.5, calculate the obovoid measurement add to the array
            self.shapeeccentric = "Obovoid"
            self.indexovoid = 0
            self.indexobovoid = -4*(self.y-0.5)
      
        return self.shapeeccentric, self.indexovoid, self.indexobovoid
    
    def blockiness(self):
        # proximal blockiness in the ratio between the proximal width and the width at mid height
        self.blockiness_prox = self.proxwidth/self.midwidth
        # distal blockiness in the ratio between the distal width and the width at mid height
        self.blockiness_dist = self.distwidth/self.midwidth
        
        return self.blockiness_prox, self.blockiness_dist

In [ ]:
# Finding distances
class Finding_Distances():
    """ To be able to make indentation measurements, the distances of each point on the boundary of the mask 
    and the centroid of the mask are measured. These distances are then sorted to travel counterclockwise over the boundary. 
    Additionally, the coordinates of the boundary are also sorted in this way.
    """
    indentation_separation = 0.02
    
    def __init__(self):
        self.instance_size = Fruit_Size(mask, pixelsize)
        self.mask = mask
    
    def mask_coordinates(self):
        # find the coordinates of the mask
        self.edge = np.argwhere(self.mask)
        self.edge[:, 0], self.edge[:, 1] = self.edge[:, 1], self.edge[:, 0].copy()
        
        return self.edge

    def edge_coordinates(self):
        # find the coordinates of the mask outline (edges) and calculate the perimeter.
        self.edges = scipy.ndimage.distance_transform_cdt(self.mask) == 1
        self.perimeter = sum(sum(self.edges))
        # The fraction of the perimeter is used to determine the distance of the indentation
        # Take 2% of perimeter to define the points to calculate proximal and distal angles
        self.perimeter_fraction = round(self.indentation_separation*self.perimeter) 
        
        # finding the coordinates (indexes) of the edge of the mask
        self.edge2 = np.argwhere(self.edges == 1)
        self.edge2[:, 0], self.edge2[:, 1] = self.edge2[:, 1], self.edge2[:, 0].copy()
        
        return self.perimeter_fraction, self.edge2
    
    def centroid(self):
        # calculate the centroid position
        self.edge = self.mask_coordinates()
        self.length = self.edge.shape[0]
        self.sum_x = np.sum(self.edge[:, 0])
        self.sum_y = np.sum(self.edge[:, 1])
        self.centroidx = self.sum_x/self.length
        self.centroidy = self.sum_y/self.length
        
        return self.centroidx, self.centroidy
    
    def edge_angle(self):
        self.perimeter_fraction, self.edge2 = self.edge_coordinates()
        self.centroidx, self.centroidy = self.centroid()
        
        self.Angle = []        
        # Using arc tangent formulas to find the angle between centroid point of the mask and the points in the outline.
        for j in range(len(self.edge2[:,1])):
            self.xdif = self.edge2[j,0] - self.centroidx
            self.ydif = self.edge2[j,1] - self.centroidy
                     
            if self.ydif<=0 and self.xdif>0:
                self.angle = np.arctan(-self.ydif/self.xdif)
            elif self.ydif<0 and self.xdif<=0:
                self.angle = np.pi - np.arctan(-self.ydif/-self.xdif)
            elif self.ydif>=0 and self.xdif<0:    
                self.angle = np.arctan(self.ydif/-self.xdif) + np.pi
            elif self.ydif>0 and self.xdif>=0:
                self.angle = 2*np.pi - np.arctan(self.ydif/self.xdif)
            self.Angle.append(self.angle)
            
        return self.Angle
    
    def edge_distances(self):
        self.perimeter_fraction, self.edge2 = self.edge_coordinates()
        self.centroidx, self.centroidy = self.centroid()
        self.Angle = self.edge_angle()
        
        self.R = []
        # Compute distances between center and mask outline
        for j in range(len(self.edge2)):
            # Distance for each point on the mask
            self.r = np.sqrt((self.edge2[j,0]-self.centroidx)**2+(self.edge2[j,1]-self.centroidy)**2) 
            self.R.append(self.r) # Array of all distances
                
        self.distances = np.column_stack((self.R, self.Angle))
        
        return self.distances
    
    def sort_coor_dist(self):
        # sort the array of the distances and the coordinates of the mask boundary by going from the right center point 
        # of the mask counterclockwise through all the points
        self.perimeter_fraction, self.edge2 = self.edge_coordinates()
        self.distances = self.edge_distances()
        self.sorted1 = np.argsort(self.distances[:, 1])
        self.sortlength = len(self.sorted1)
        
        self.sorted_distances = np.copy(self.distances)
        self.sorted_coords = np.copy(self.edge2)
        
        for j in range(self.sortlength):
            self.sorted_distances[j,:] = self.distances[self.sorted1[j], :]
            self.sorted_coords[j,:] = self.edge2[self.sorted1[j], :]
        
        return self.sorted_distances, self.sorted_coords

In [ ]:
# Fruit End Shape
class Fruit_End_Shape():
    """ The fruit end shape can only be calculated in the L-En-R mask (where the indentation can be visible). 
    All fruit end shape measurements are calculated for both the proximal and distal end of the tomato. 
    Three measurements are performed to calculate the fruit end shape: the angle, the shoulderheight and the resemblance to a 
    heart.
    """
    
    def __init__(self):
        self.instance_shape = Fruit_Shape()
        self.widestindex, self.y = self.instance_shape.find_y()
        self.instance_size = Fruit_Size(mask, pixelsize)
        self.mask = mask
        self.width, self.maxwidth = self.instance_size.max_width()
        self.height, self.maxheight = self.instance_size.max_height()
        self.firstypoint, self.lastypoint, self.firstxpoint, self.lastxpoint = self.instance_size.extreme_points()
        self.instance_dist = Finding_Distances()
        self.sorted_distances, self.sorted_coords = self.instance_dist.sort_coor_dist()
        self.perimeter_fraction, self.edge2 = self.instance_dist.edge_coordinates()
        self.centroidx, self.centroidy = self.instance_dist.centroid()

    def Taperness(self):
    # calculating taperness
        self.wid1 = self.width[self.firstypoint:self.widestindex]
        self.w1 = np.mean(self.wid1)
        self.wid2 = self.width[self.widestindex:self.lastypoint]
        self.w2 = np.mean(self.wid2)
        self.taperness = 1-(self.w2/self.maxwidth)+(self.w1/self.maxwidth)
        
        return self.taperness
    
    def indent_distances(self):
        # generating an ellipsoid that is somewhat smaller than the tomato's ellipse 
        # so that all the points from the tomato will fall outside the ellipse, except for the intentation      
        self.ellipsex = ((self.maxwidth/2)*0.9)*np.cos(self.sorted_distances[:,1]) + self.centroidx #ellipsoid at 90% of tomato width
        self.ellipsey = ((self.maxheight/2)*0.9)*np.sin(self.sorted_distances[:,1]) + self.centroidy #and 90% of tomato height
                
        self.sortlength = self.sorted_coords.shape[0]
        self.half_sortlength = int(self.sortlength/2)
                
        self.p_dist = self.sorted_distances[1,0]
        self.d_dist = self.sorted_distances[self.half_sortlength,0]
        for m in range(self.sortlength):
            self.distance = self.sorted_distances[m,0]
            self.el_distance = np.sqrt((self.ellipsex[m]-self.centroidx)**2+(self.ellipsey[m]-self.centroidy)**2)
            
            if self.distance < self.el_distance and m <self.half_sortlength and self.distance<self.p_dist:
                self.p_dist = self.distance
            elif self.distance < self.el_distance and self.half_sortlength<m<self.sortlength and self.distance<self.d_dist:
                self.d_dist = self.distance
                
        return self.p_dist, self.d_dist
    
    def prox_point(self):
        self.P = []
        self.Ap = []
        self.Bp = []
        
        self.p_dist, self.d_dist = self.indent_distances()
        self.sortlength = self.sorted_coords.shape[0]

        self.dist_ = self.sorted_distances[:,0]
        self.dist_list = self.dist_.tolist()
                
        #If there is a proximal distance found, do calculations
        if self.p_dist!=self.sorted_distances[1,0]:
            self.p_index = self.dist_list.index(self.p_dist)
            self.p_coord = self.sorted_coords[self.p_index,:]
            self.p_slope = (self.centroidx-self.p_coord[0])/(self.p_coord[1]-self.centroidy) # negative reciprocal of slope of line Centroid to P
            self.p_start = self.p_coord[1]-self.p_slope*self.p_coord[0] # starting point of line through P
                            
            for m in range(self.sortlength):
                if m < self.p_index:
                    self.edge_part = self.edge2[m:self.p_index,:]
                    if self.p_index-m == self.perimeter_fraction: #p_index-m: amount of points (and thus length) between point p and m
                        self.first_coord = self.sorted_coords[m,:]
                        self.dx1 = abs(self.first_coord[0]-self.p_coord[0])
                        self.dy1 = abs(self.first_coord[1]-self.p_coord[1])
                elif m > self.p_index: 
                    self.edge_part = self.edge2[self.p_index:m,:]
                    if m-self.p_index == self.perimeter_fraction:
                        self.second_coord = self.sorted_coords[m,:]
                        self.dx2 = abs(self.second_coord[0]-self.p_coord[0])
                        self.dy2 = abs(self.second_coord[1]-self.p_coord[1])
                        
                if self.sorted_coords[m,1] == round(self.p_slope*self.sorted_coords[m,0]+self.p_start):
                    self.P.append(self.sorted_coords[m,0])
                    self.P.append(self.sorted_coords[m,1])
                    self.P.append(m)
                            
            self.p_angle = np.arctan(self.dx1/self.dy1)+ np.arctan(self.dx2/self.dy2)
                 
            self.Ap.append(self.P[0])
            self.Ap.append(self.P[1])
            self.Ap_index = self.P[2]
                
            self.Bp.append(self.P[len(self.P)-3])
            self.Bp.append(self.P[len(self.P)-2])
            self.Bp_index = self.P[len(self.P)-1]
        else:
            self.p_angle = 0
            self.p_index = 0
            self.Ap_index = 0
            self.Bp_index = 0
            
        return self.p_angle, self.p_index, self.Ap, self.Ap_index, self.Bp, self.Bp_index
        
    def prox_endshape(self):
        self.p_dist, self.d_dist = self.indent_distances()
        self.taperness = self.Taperness()
        self.p_angle, self.p_index, self.Ap, self.Ap_index, self.Bp, self.Bp_index = self.prox_point()
        if self.p_dist!=self.sorted_distances[1,0]:
            if len(self.Ap)>0 and len(self.Bp)>0:
                self.hp1=0
                for m in range(self.Ap_index, self.p_index):
                    self.ypoint = (self.Bp[1]/self.Ap[1])*m+(self.Bp[1]*(1-(self.Bp[0]/self.Ap[1])))
                    self.hp = self.sorted_coords[m,1]-self.ypoint
                    if self.hp > self.hp1:
                        self.hp1 = self.hp
                self.hp2=0
                for m in range (self.p_index, self.Bp_index):
                    self.ypoint = (self.Bp[1]/self.Ap[1])*m+(self.Bp[1]*(1-(self.Bp[0]/self.Ap[1])))
                    self.hp = self.sorted_coords[m,1]-self.ypoint
                    if self.hp > self.hp2:
                        self.hp2 = self.hp
            
            # calculating shoulderheight
            self.shoulderheight_p = (self.hp1+self.hp2)/(2*self.maxheight)   
            self.heartp = 0.25*((1-self.y)*self.taperness)+20*self.shoulderheight_p
        
        return self.shoulderheight_p, self.heartp, self.p_angle
            
    def dist_point(self):
        self.D = []
        self.Ad = []
        self.Bd = []
        
        self.p_dist, self.d_dist = self.indent_distances()
        self.sortlength = self.sorted_coords.shape[0]
        
        self.dist_ = self.sorted_distances[:,0]
        self.dist_list = self.dist_.tolist()
        
        if self.d_dist!=self.sorted_distances[self.half_sortlength,0]:  
            self.d_index = self.dist_list.index(self.d_dist)
            self.d_coord = self.sorted_coords[self.d_index,:]
            self.d_slope = (self.centroidx-self.d_coord[0])/(self.d_coord[1]-self.centroidy) # negative reciprocal of slope of line Centroid to D
            self.d_start = self.d_coord[1]-self.d_slope*self.d_coord[0] # starting point of line through P
                      
            for m in range(self.sortlength): 
                if m < self.d_index:
                    self.edge_part = self.edge2[m:self.d_index,:]
                    if self.d_index-m == self.perimeter_fraction:
                        self.first_coord = self.sorted_coords[m,:]
                        self.dx1 = abs(self.first_coord[0]-self.p_coord[0])
                        self.dy1 = abs(self.first_coord[1]-self.p_coord[1])
                elif m > self.d_index: 
                    self.edge_part = self.edge2[self.d_index:m,:]
                    if m-self.d_index == self.perimeter_fraction:
                        self.second_coord = self.sorted_coords[m,:]
                        self.dx2 = abs(self.second_coord[0]-self.p_coord[0])
                        self.dy2 = abs(self.second_coord[1]-self.p_coord[1])
                        
                if self.sorted_coords[m,1] == round(self.d_slope*self.sorted_coords[m,0]+self.d_start):
                    self.D.append(self.sorted_coords[m,0])
                    self.D.append(self.sorted_coords[m,1])
                    self.D.append(m)
                    
            self.d_angle = np.arctan(self.dx1/self.dy1)+np.arctan(self.dx2/self.dy2)
                    
            self.Ad.append(self.D[0])
            self.Ad.append(self.D[1])
            self.Ad_index = self.D[2]
                
            self.Bd.append(self.D[len(self.D)-3])
            self.Bd.append(self.D[len(self.D)-2])
            self.Bd_index = self.D[len(self.D)-1]
        else:
            self.d_angle = 0
            self.d_index = 0
            self.Ad_index = 0
            self.Bd_index = 0
            
        return self.d_angle, self.d_index, self.Ad, self.Ad_index, self.Bd, self.Bd_index
    
    def dist_endshape(self):
        self.p_dist, self.d_dist = self.indent_distances()
        self.taperness = self.Taperness()
        self.d_angle, self.d_index, self.Ad, self.Ad_index, self.Bd, self.Bd_index = self.dist_point()
        
        self.sortlength = self.sorted_coords.shape[0]
        self.half_sortlength = int(self.sortlength/2)
        
        if self.d_dist!=self.sorted_distances[self.half_sortlength,0]:  
            if len(self.Ad)>0 and len(self.Bd)>0:
                self.hd1=0
                for m in range(self.Ad_index, self.d_index):
                    self.ypoint = (self.Bd[1]/self.Ad[1])*m+(self.Bd[1]*(1-(self.Bd[0]/self.Ad[1])))
                    self.hd = self.sorted_coords[m,1]-self.ypoint
                    if self.hd > self.hd1:
                        self.hd1 = self.hd
                self.hd2=0
                for m in range (self.d_index, self.Bd_index):
                    self.ypoint = (self.Bd[1]/self.Ad[1])*m+(self.Bd[1]*(1-(self.Bd[0]/self.Ad[1])))
                    self.hd = self.sorted_coords[m,1]-self.ypoint
                    if self.hd > self.hd2:
                        self.hd2 = self.hd
            
            # calculating shoulderheight
            self.shoulderheight_d = (self.hd1+self.hd2)/(2*self.maxheight)
            self.heartd = 0.25*((1-self.y)*self.taperness)+20*self.shoulderheight_d
    
        return self.shoulderheight_d, self.heartd, self.d_angle

In [ ]:
# Fruit Shape Resemblance
class Fruit_Shape_Resemblance():
    """ Fruit shape resemblance is calculated in three measurements: Circularity (Compactness), Ellipsoid (CompactnessEllipse)
    and Rectangularity. A value closer to one indicates more resemblance.
    """
    
    def __init__(self):
        self.instance_size = Fruit_Size(mask, pixelsize)
        self.mask = mask
        self.width, self.maxwidth = self.instance_size.max_width()
        self.height, self.maxheight = self.instance_size.max_height()
        self.cmheight = self.instance_size.cm_height()
        self.cmwidth = self.instance_size.cm_width()
        self.cmarea = self.instance_size.cm_area()
        self.firstypoint, self.lastypoint, self.firstxpoint, self.lastxpoint = self.instance_size.extreme_points()
        self.instance_shape = Fruit_Shape()
        self.shapeindex = self.instance_shape.shape_index()
        self.instance_dist = Finding_Distances()
        self.sorted_distances, self.sorted_coords = self.instance_dist.sort_coor_dist()
        
    def compactness(self):
    # circularity (compactness) and ellipsoid (compactnessellipse) resemblance measurements    
        self.compactness = 4*self.cmarea/(np.pi*(self.cmheight**2))
        self.compactnessellipse = 4*self.cmarea/(np.pi*self.cmheight*self.cmwidth)
        
        return self.compactness, self.compactnessellipse

    def rectangular(self):
        # rectangularity is determined by the ratio between an inner (Sin) and outer (Sout) rectangle.
        # The inner rectangle is determined by a diagonal line from the outer rectangle and finding the inner intersection.
        self.rect_diag_start = self.firstypoint-self.shapeindex*self.firstxpoint
        self.Diag = []
        for m in range(self.sorted_coords.shape[0]):                    
            if self.sorted_coords[m,1] == round(self.shapeindex*self.sorted_coords[m,0]+self.rect_diag_start):
                self.Diag.append(self.sorted_coords[m,0])
                self.Diag.append(self.sorted_coords[m,1])
                self.Diag.append(m)
        
        self.Sout = self.maxwidth*self.maxheight
        self.Sin = (self.Diag[len(self.Diag)-3]-self.Diag[0])*(self.Diag[len(self.Diag)-2]-self.Diag[1])
        self.rectangular = self.Sin/self.Sout
        
        return self.rectangular

In [ ]:
filelist = os.listdir(TEST_IMAGE_DIR) # dir is your directory path
number_files = len(filelist)
print(number_files)

for n in range(0,number_files):
    file_names = next(os.walk(TEST_IMAGE_DIR))[2]
    image_name = os.path.join(TEST_IMAGE_DIR, file_names[n])
    image = skimage.io.imread(image_name, plugin='matplotlib')
    file_name = file_names[n]

    results = model.detect([image], verbose=1)
    r = results[0]
    masks = r["masks"]
    roishape = r["rois"].shape
    classids = r["class_ids"]

    # Determine if the image contains a grid or not
    ImageType = Image_Type(image_name)
    
    
    # If it is a grid image, calculate the scale for the image
    if ImageType == "Grid":
        pixelsize = Grid_Scale(ImageType, image_name)
    
    else:
        if 1 in classids:
            ImageType = "Ruler"
            print(ImageType)
            
            pixelsize = Ruler_Scale(classids, roishape, masks)
        else: 
            ImageType = "No indicator" 
            print("Image does not contain a scale indicator")
        
    if ImageType == "Ruler" or ImageType == "Grid":
        # loop over of the detected object's bounding boxes and masks
        for i in range(0, len(classids)):
            # extract the class ID, classnames and mask for the current detection
            classID = classids[i]
            classnames = class_names[classID]
            mask = masks[:, :, i]
            instance = file_name + '_' + classnames
            
            print(classID)
            print(classnames)
            print(instance)
            
            FileName.append(image_name)
            TomatoView.append(class_names[classID])
        
            instance_size = Fruit_Size(mask, pixelsize)
            instance_shape = Fruit_Shape()
            instance_resemblance = Fruit_Shape_Resemblance()
            instance_endshape = Fruit_End_Shape()
            instance_dist = Finding_Distances()
        
            height, maxheight = instance_size.max_height()
            cmheight = instance_size.cm_height()
            MaxHeight.append(maxheight)        
            CmHeight.append(cmheight)
            
            width, maxwidth = instance_size.max_width()
            cmwidth = instance_size.cm_width()
            MaxWidth.append(maxwidth)
            CmWidth.append(cmwidth)
            
            area = instance_size.pixel_area()
            cmarea = instance_size.cm_area()
            Area.append(area)
            CmArea.append(cmarea)
                   
            shapeindex = instance_shape.shape_index()
            ShapeIndex.append(shapeindex)
            shapetriangle = instance_shape.shape_triangle()
            ShapeTriangle.append(shapetriangle)
        
            shapeeccentric, indexovoid, indexobovoid = instance_shape.eccentric()
            widestindex, y = instance_shape.find_y()
            if y > 0.5:
                IndexOvoid.append(indexovoid)
                IndexObovoid.append(None)
            else:
                IndexObovoid.append(indexobovoid)
                IndexOvoid.append(None)
            ShapeEccentric.append(shapeeccentric)
            
            asymmetryhorizontal, asymmetryvertical = instance_shape.asymmetry()
            AsymmetryHorizontal.append(asymmetryhorizontal)
            AsymmetryVertical.append(asymmetryvertical)
            
            blockiness_prox, blockiness_dist = instance_shape.blockiness()
            BlockinessProx.append(blockiness_prox)
            BlockinessDist.append(blockiness_dist)
            
            compactness, compactnessellipse = instance_resemblance.compactness()
            Compactness.append(compactness)
            CompactnessEllipse.append(compactnessellipse)
            
            rectangular = instance_resemblance.rectangular()
            Rectangular.append(rectangular)
                
            if classnames == "L-En-R":
                sorted_distances, sorted_coords = instance_dist.sort_coor_dist()
                p_dist, d_dist = instance_endshape.indent_distances()
                sortlength = sorted_coords.shape[0]
                half_sortlength = int(sortlength/2)
                
                if p_dist!=sorted_distances[1,0]:
                    shoulderheight_p, heartp, p_angle = instance_endshape.prox_endshape()
                    ShoulderheightProx.append(shoulderheight_p)
                    HeartP.append(heartp)
                    AngleProx.append(p_angle)
                else:
                    ShoulderheightProx.append(None)
                    HeartP.append(None)
                    AngleProx.append(None)
                    
                if d_dist!=sorted_distances[half_sortlength,0]:
                    shoulderheight_d, heartd, d_angle = instance_endshape.dist_endshape()
                    ShoulderheightDist.append(shoulderheight_d)
                    HeartD.append(heartd)
                    AngleDist.append(d_angle)
                else:
                    ShoulderheightDist.append(None)
                    HeartD.append(None)
                    AngleDist.append(None)    
            else:
                ShoulderheightProx.append(None) 
                ShoulderheightDist.append(None)
                HeartP.append(None)
                HeartD.append(None)
                AngleProx.append(None)
                AngleDist.append(None)

Sizes = np.column_stack((FileName, TomatoView, MaxHeight, CmHeight, MaxWidth, CmWidth, Area, CmArea, ShapeIndex, ShapeTriangle, ShapeEccentric, IndexObovoid, IndexOvoid, AsymmetryHorizontal, AsymmetryVertical, AngleProx, AngleDist, BlockinessProx, BlockinessDist, ShoulderheightProx, ShoulderheightDist, Compactness, CompactnessEllipse, HeartP, HeartD, Rectangular))
Column_Names = ["File Name", "Type of Tomato Section", "Maximum Height (pixels)", "Maximum Height (cm)", "Maximum Width (pixels)", "Maximum Width (cm)", "Area (pixels)", "Area (cm)", "Shape Index", "Shape Triangle", "Eccentric", "Obovoid", "Ovoid", "Horizontal Asymmetry", "Vertical Asymmetry", "Proximal Angle (rad)", "Distal Angle (rad)", "Proximal Blockiness", "Distal Blockiness", "Proximal Shoulderheight", "Distal Shoulderheight", "Circular", "Ellipsoid", "Proximal Heart", "Distal Heart", "Rectangular"]

In [ ]:
#exporting to excel file in d/m/y format
import pandas as pd
from datetime import datetime

df = pd.DataFrame(Sizes)
df.columns = Column_Names
now = datetime.now()
fdate = now.strftime('%Y%m%d')
ftime = now.strftime('%H%M')

excelname = '/mnt/MeasurementResults/ShapeSizeMeasurements_'+fdate+'T'+ftime+'.xlsx'
df.to_excel(excelname)